In [ ]:
# filepath: [start.ipynb](http://_vscodecontentref_/0)
import subprocess
import threading

def leer_salida(proc):
    for line in proc.stdout:
        print(line, end="")

# Preguntar al usuario el modo
while True:
    modo = input("¿Cómo quieres iniciar el servidor? Escribe 'mods' o 'vanilla': ").strip().lower()
    if modo in ["mods", "vanilla"]:
        break
    print("Opción no válida. Escribe 'mods' o 'vanilla'.")

# Si elige mods, preguntar Forge o Fabric
modloader = None
if modo == "mods":
    while True:
        modloader = input("¿Qué modloader quieres usar? Escribe 'forge' o 'fabric': ").strip().lower()
        if modloader in ["forge", "fabric"]:
            break
        print("Opción no válida. Escribe 'forge' o 'fabric'.")

# --- Agrega aquí el bloque de selección de Java ---
# Leer la versión de Minecraft desde VERSION.txt
with open("server/VERSION.txt") as f:
    version_mc = f.read().strip()

# Diccionario de versiones recomendadas de Java
java_por_mc = {
    "1.16": "/usr/lib/jvm/java-8-openjdk-amd64/bin/java",
    "1.17": "/usr/lib/jvm/java-16-openjdk-amd64/bin/java",
    "1.18": "/usr/lib/jvm/java-17-openjdk-amd64/bin/java",
    "1.19": "/usr/lib/jvm/java-17-openjdk-amd64/bin/java",
    "1.20": "/usr/lib/jvm/java-17-openjdk-amd64/bin/java",
}
java_cmd = "java"
for v, path in java_por_mc.items():
    if version_mc.startswith(v):
        java_cmd = path
        break

# Lanzar el servidor con el modo elegido y la ruta de Java
args = ["python3", "server/server.py", modo]
if modloader:
    args.append(modloader)
args.extend(["--java", java_cmd])

server_process = subprocess.Popen(
    args,
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

hilo_salida = threading.Thread(target=leer_salida, args=(server_process,))
hilo_salida.daemon = True
hilo_salida.start()

try:
    while True:
        if server_process.poll() is not None:
            print("El servidor se ha detenido.")
            break
        comando = input("Comando para el servidor (stop para salir): ")
        if comando.strip().lower() in ["exit", "stop"]:
            try:
                server_process.stdin.write("stop\n")
                server_process.stdin.flush()
            except Exception:
                pass
            break
        try:
            server_process.stdin.write(comando + "\n")
            server_process.stdin.flush()
        except Exception:
            print("No se pudo enviar el comando. El servidor puede haberse detenido.")
            break
except KeyboardInterrupt:
    try:
        server_process.stdin.write("stop\n")
        server_process.stdin.flush()
    except Exception:
        pass

server_process.wait()
print("Servidor detenido.")